# Run py12box model
Run the AGAGE 12-box model

In [ ]:
%load_ext autoreload
%autoreload 2

# Import some standard libraries
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from pandas import read_csv

# Import from this package
from py12box.model import Model
from py12box import get_data


In [ ]:
# Initialise the model
mod = Model("CFC-11", get_data("example"))


In [ ]:
# Run model
mod.run()

In [ ]:
plt.plot(mod.time, mod.mf[:, 0])
plt.plot(mod.time, mod.mf[:, 3])
plt.ylabel("%s (pmol mol$^{-1}$)" % mod.species)
plt.show()

In [ ]:
plt.plot(mod.emissions.sum(axis = 1).cumsum())
plt.plot(mod.burden.sum(axis = 1))
plt.plot(mod.losses["other"].sum(axis = 1).cumsum())

In [ ]:
plt.plot(mod.lifetimes["global_total"])